## MACHINE TRANSLATION

In [ ]:
# kipkoh=[]
# while(1):
#   kipkoh.append('1')

In [ ]:
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
import string
import re
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from keras import optimizers

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 1.2 MB 11.1 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 65 kB 3.7 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=9937ac12ed65e915711258344955a24b3b0b9cdb0dccf2670ae458a7da3fae13
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [ ]:
# Imports
import pandas as pd

import googletrans
from googletrans import Translator


In [ ]:
! ls

drive  sample_data


### Translate English articles to Swahili articles

In [ ]:
pd.set_option('max_colwidth', 300)
 
# how to get the supported language and their corresponing code
lang_df = pd.DataFrame.from_dict(googletrans.LANGUAGES,  orient='index', columns=['Language'])
print(len(googletrans.LANGUAGES))
lang_df.head(3)


107


,Language
af,afrikaans
sq,albanian
am,amharic


In [ ]:
# find the code for english and swahili
lang_df[lang_df.Language.isin(['english', 'swahili'])]

,Language
en,english
sw,swahili


In [ ]:
# os.environ["gdrive_path"]='/content/drive/MyDrive/colab/MT_TRANSALATOR/'
!ls /content/drive/MyDrive/MT_TRANSALATOR/


ls: cannot access '/content/drive/MyDrive/MT_TRANSALATOR/': No such file or directory


In [ ]:
filename = '/content/drive/MyDrive/colab/MT_TRANSALATOR/english.en'
file = open(filename, mode='r') # 'r' is to read
en = pd.DataFrame(file)
en = en.rename(columns={0:'EnglishText'})
en.head()

,EnglishText
0,Article 2\n
1,"Netflix advertisement for the movie Santana, to be premiered on 28 August.\n"
2,Screengrab.\n
3,Image text: I want Dias Santana and his brother dead\n
4,"From August 28, an Angolan production will appear for the first time in Netflix's catalogue.\n"


In [ ]:
filename1 = '/content/drive/MyDrive/colab/MT_TRANSALATOR/swahili.sw'
file1 = open(filename1, mode='r') # 'r' is to read
sw = pd.DataFrame(file1)
sw = sw.rename(columns={0:"SwahiliText"})
sw.head()

,SwahiliText
0,Makala 2\n
1,"Tangazo la Netflix kwa ajili ya filamu ya Santana, itakayokwenda hewani mnamo Agosti 28.\n"
2,Picha imepigwa kwenye filamu.\n
3,Maneno yanayoonekana yanasema: Ninataka Dias Santana na kaka yake wauawe\n
4,"Kuanzia Agosti 28, filamu iliyozalishwa Angola itaonekana kwa mara ya kwanza katika orodha ya Netflix.\n"


In [ ]:
en['SwahiliText']=sw.SwahiliText
en.head()

,EnglishText,SwahiliText
0,Article 2\n,Makala 2\n
1,"Netflix advertisement for the movie Santana, to be premiered on 28 August.\n","Tangazo la Netflix kwa ajili ya filamu ya Santana, itakayokwenda hewani mnamo Agosti 28.\n"
2,Screengrab.\n,Picha imepigwa kwenye filamu.\n
3,Image text: I want Dias Santana and his brother dead\n,Maneno yanayoonekana yanasema: Ninataka Dias Santana na kaka yake wauawe\n
4,"From August 28, an Angolan production will appear for the first time in Netflix's catalogue.\n","Kuanzia Agosti 28, filamu iliyozalishwa Angola itaonekana kwa mara ya kwanza katika orodha ya Netflix.\n"


In [ ]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 80 kB 4.0 MB/s 


In [ ]:
en_df=en

In [ ]:
en_df.shape

(3815, 2)

In [ ]:
en_df.head()

,EnglishText,SwahiliText
0,Article 2\n,Makala 2\n
1,"Netflix advertisement for the movie Santana, to be premiered on 28 August.\n","Tangazo la Netflix kwa ajili ya filamu ya Santana, itakayokwenda hewani mnamo Agosti 28.\n"
2,Screengrab.\n,Picha imepigwa kwenye filamu.\n
3,Image text: I want Dias Santana and his brother dead\n,Maneno yanayoonekana yanasema: Ninataka Dias Santana na kaka yake wauawe\n
4,"From August 28, an Angolan production will appear for the first time in Netflix's catalogue.\n","Kuanzia Agosti 28, filamu iliyozalishwa Angola itaonekana kwa mara ya kwanza katika orodha ya Netflix.\n"


In [ ]:
en_df.columns

Index(['EnglishText', 'SwahiliText'], dtype='object')

In [ ]:
en_df.rename(columns={'EnglishText':'source_sentence','SwahiliText':'target_sentence'},inplace=True)

In [ ]:
df1=pd.read_csv('/content/drive/MyDrive/colab/MT_TRANSALATOR/data.csv')
df1.head()

,source_sentence,target_sentence
0,“ Look !,“ Tazama !
1,I Am Making All Things New ”,Mimi Ninafanya Vitu Vyote Kuwa Vipya ”
2,The above is a promise from God that has been adopted as the title of an inspiring brochure .,Iliyopo juu ni ahadi itokayo kwa Mungu ambayo imechaguliwa iwe kichwa cha broshua yenye kuvuvia tumaini .
3,( Revelation 21 : 5 ) What does that promise mean for us ?,"( Ufunuo 21 : 5 , NW ) Ahadi hiyo yamaanisha nini kwetu ?"
4,"How will it be fulfilled , and when ?","Itatimizwa jinsi gani , na wakati gani ?"


In [ ]:
import os
source_language = "en"
target_language = "sw" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!sudo mkdir -p "/content/drive/MyDrive/colab/MLT/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/MyDrive/colab/MLT/%s-%s-%s" % (source_language, target_language, tag)


In [ ]:
!echo $gdrive_path

/content/drive/MyDrive/colab/MLT/en-sw-baseline


In [ ]:
!ls /content/drive/MyDrive/colab/MLT

en-sw-baseline


In [ ]:

# Install opus-tools
! pip install opustools-pkg

In [ ]:
# # Downloading our corpus
# ! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# # extract the corpus file
# ! gunzip JW300_latest_xml_$src-$tgt.xml.gz

In [ ]:
d2=df1.sample(n=60000,random_state=42)

In [ ]:
df=df1.copy()

In [ ]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2021-10-22 07:00:21--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.03s   

2021-10-22 07:00:22 (8.69 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2021-10-22 07:00:22--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-sw.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 2

In [ ]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [ ]:
# filename1 = 'test.en-any.en'
# file1 = open(filename1, mode='r') # 'r' is to read
# test1 = pd.DataFrame(file1)
# test1=test1.sample(n=500,random_state=42)
# test1.to_csv('test.en-any.en', index=False)

In [ ]:
# filename1 = 'test.en-any.en'
# file1 = open(filename1, mode='r') # 'r' is to read
# test1 = pd.DataFrame(file1)
# test1.shape

####Pre-processing and export
It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [ ]:
import numpy as np
# drop duplicate translations
df_pp = df.drop_duplicates()

#drop empty lines (alp)
df_pp['source_sentence'].replace('', np.nan, inplace=True)
df_pp['target_sentence'].replace('', np.nan, inplace=True)
df_pp.dropna(subset=['source_sentence'], inplace=True)
df_pp.dropna(subset=['target_sentence'], inplace=True)

# drop conflicting translations
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12

In [ ]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)


# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*


==> train.en <==
His desire is that you attain to “ the real life ” — everlasting life on a paradise earth .
BE POSITIVE : “ All the days of the afflicted one are bad , but the one with a cheerful heart has a continual feast . ”
Peter reached the gate and was refused entrance .
Sudden sharp sounds reverberated like rifle shots .
The record tells us : “ He opened the scroll and found the place where it was written : ‘ Jehovah ’ s spirit is upon me , because he anointed me to declare good news to the poor , he sent me forth to preach a release to the captives and a recovery of sight to the blind , to send the crushed ones away with a release . ’
( Num . 25 : 1 , 2 ) Jehovah afflicted the wrongdoers with a death-dealing scourge .
The Bible says that God “ heals the brokenhearted ; he binds up their wounds . ”
Cristina , Italy : “ When a problem seems beyond my ability to solve , I pray to Jehovah , leaving it in his hands .
▪ How did Jesus ’ followers view the secular authorities ? — Roma

In [ ]:
#Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
# !rm -rf joeynmt

Cloning into 'joeynmt'...
remote: Enumerating objects: 3224, done.
remote: Counting objects: 100% (273/273), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 3224 (delta 157), reused 206 (delta 134), pack-reused 2951
Receiving objects: 100% (3224/3224), 8.17 MiB | 15.43 MiB/s, done.
Resolving deltas: 100% (2186/2186), done.


In [ ]:
!pip install joeynmt

     |████████████████████████████████| 84 kB 2.2 MB/s 
     |████████████████████████████████| 392 kB 32.7 MB/s 
     |████████████████████████████████| 15.3 MB 106 kB/s 
     |████████████████████████████████| 735.5 MB 11 kB/s 
     |████████████████████████████████| 7.1 MB 12.5 MB/s 
     |████████████████████████████████| 90 kB 8.5 MB/s 
     |████████████████████████████████| 596 kB 48.3 MB/s 
     |████████████████████████████████| 106 kB 48.7 MB/s 
     |████████████████████████████████| 246 kB 45.2 MB/s 
     |████████████████████████████████| 743 kB 44.1 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
  Created wheel for wrapt: filename=wrapt-1.11.1-cp37-cp37m-linux_x86_64.whl size=68437 sha256=2bd7b855ab55112f1e2b10ecb6dc4109422a3e550450d790bb8487a901373d01
  Stored in directory: /root/.cache/pip/wheels/4e/58/9d/da8bad4545585ca52311498ff677647c95c7b690b3040171f8
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: t

In [ ]:
! cd joeynmt;


#### Preprocessing the Data into Subword BPE Tokens
One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization (Sennrich, 2015) .

It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages (Sennrich, 2019) (Martinus, 2019)

Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by (Sennrich, 2019). You do not need to change anything. Simply running the below will be suitable.

In [ ]:
# path='/content/drive/MyDrive/colab/MT_TRANSALATOR/'

In [ ]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

In [ ]:

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

In [ ]:
# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path


bpe.codes.4000	dev.en	     test.bpe.sw     test.sw	   train.en
dev.bpe.en	dev.sw	     test.en	     train.bpe.en  train.sw
dev.bpe.sw	test.bpe.en  test.en-any.en  train.bpe.sw


In [ ]:
#Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.*  "$gdrive_path"
! cp dev.*  "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path

/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [ ]:
!ls joeynmt

benchmarks.md	    data	     joeynmt	     README.md	       setup.py
CODE_OF_CONDUCT.md  docs	     joey-small.png  requirements.txt  test
configs		    joey_demo.ipynb  LICENSE	     scripts


In [ ]:
# Create that vocab using build_vocab
# ! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

In [ ]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.sw     test.en-any.en.1  train.bpe.sw
dev.bpe.en	dev.sw	     test.en	     test.sw	       train.en
dev.bpe.sw	test.bpe.en  test.en-any.en  train.bpe.en      train.sw



####Creating the JoeyNMT Config
JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

We used Transformer architecture
We set our dropout to reasonably high: 0.3 (recommended in (Sennrich, 2019))
Things worth playing with:

The batch size (also recommended to change for low-resourced languages)
The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
The decoder options (beam_size, alpha)
Evaluation metrics (BLEU versus Crhf4)

In [ ]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
# gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_filtered_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "/content/joeynmt/data/{name}/train.bpe"
    dev:   "/content/joeynmt/data/{name}/dev.bpe"
    test:  "/content/joeynmt/data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "/content/joeynmt/data/{name}/vocab.txt"
    trg_vocab: "/content/joeynmt/data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "/content/joeynmt/models/{name}_filtered_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path="joeynmt", source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_filtered_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [ ]:
!readlink -f train

/content/train


In [ ]:
!ls joeynmt

benchmarks.md	    data	     joeynmt	     README.md	       setup.py
CODE_OF_CONDUCT.md  docs	     joey-small.png  requirements.txt  test
configs		    joey_demo.ipynb  LICENSE	     scripts


In [ ]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python -m joeynmt train /content/joeynmt/configs/transformer_filtered_$src$tgt.yaml

2021-10-22 07:41:05,503 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-10-22 07:41:05,581 - INFO - joeynmt.data - Loading training data...
2021-10-22 07:41:45,126 - INFO - joeynmt.data - Building vocabulary...
2021-10-22 07:41:45,156 - INFO - joeynmt.data - Loading dev data...
2021-10-22 07:41:45,244 - INFO - joeynmt.data - Loading test data...
2021-10-22 07:41:45,290 - INFO - joeynmt.data - Data loaded.
2021-10-22 07:41:45,291 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-10-22 07:41:45,817 - INFO - joeynmt.model - Enc-dec model built.
2021-10-22 07:41:50,509 - INFO - joeynmt.training - Total params: 11197184
2021-10-22 07:41:50,514 - WARNING - joeynmt.training - `keep_last_ckpts` option is outdated. Please use `keep_best_ckpts`, instead.
2021-10-22 07:41:58,215 - INFO - joeynmt.helpers - cfg.name                           : ensw_filtered_transformer
2021-10-22 07:41:58,216 - INFO - joeynmt.helpers - cfg.data.src                       : en
2021-10-22 